In [39]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import joblib
import warnings
warnings.filterwarnings('ignore')



In [40]:
train_df = pd.read_excel('../Data/train.xlsx')
val_df = pd.read_excel('../Data/validation.xlsx')
test_df = pd.read_excel('../Data/test.xlsx')

In [41]:

y_train = train_df['Treatment_Type']
X_train = train_df.drop(columns=['Treatment_Type'])

y_val = val_df['Treatment_Type']
X_val = val_df.drop(columns=['Treatment_Type'])

y_test = test_df['Treatment_Type']
X_test = test_df.drop(columns=['Treatment_Type'])

for col in X_train.columns:
    if pd.api.types.is_datetime64_any_dtype(X_train[col]):
        X_train.drop(columns=[col], inplace=True)
        X_val.drop(columns=[col], inplace=True)
        if col in X_test.columns:
            X_test.drop(columns=[col], inplace=True)

In [42]:
imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_val = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

In [43]:
n_estimators_options = [50, 100]
max_depth_options = [5, 10]
criterion_options = ['gini', 'entropy']

In [44]:
for n in n_estimators_options:
    for depth in max_depth_options:
        for crit in criterion_options:
            model = RandomForestClassifier(n_estimators=n, max_depth=depth, criterion=crit, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)

            acc = accuracy_score(y_val, y_pred)
            prec = precision_score(y_val, y_pred, average='weighted')
            rec = recall_score(y_val, y_pred, average='weighted')
            f1 = f1_score(y_val, y_pred, average='weighted')
            cm = confusion_matrix(y_val, y_pred)

            results.append({
                'n_estimators': n,
                'max_depth': depth,
                'criterion': crit,
                'Accuracy': acc,
                'Precision': prec,
                'Recall': rec,
                'F1 Score': f1,
                'Confusion Matrix': cm
            })

In [45]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='F1 Score', ascending=False).reset_index(drop=True)
print("Resultados del experimento con Random Forest:")
print(results_df)

Resultados del experimento con Random Forest:
    n_estimators  max_depth criterion  accuracy  Accuracy  Precision  \
0            100         10   entropy       NaN  0.262500   0.262710   
1            100          5   entropy       NaN  0.262500   0.261186   
2             50         10      gini       NaN  0.256250   0.254148   
3            100         10      gini       NaN  0.247500   0.247278   
4            100          5      gini       NaN  0.250625   0.250984   
5             50         10   entropy       NaN  0.245625   0.244808   
6             50          5   entropy       NaN  0.245000   0.243614   
7             50          5      gini       NaN  0.243125   0.240978   
8             50          5      gini  0.246875       NaN        NaN   
9             50          5   entropy  0.240625       NaN        NaN   
10            50         10      gini  0.260000       NaN        NaN   
11            50         10   entropy  0.242500       NaN        NaN   
12           100  

In [46]:
best_params = results_df.loc[0]
print("\nMejores parámetros:")
print(best_params)


Mejores parámetros:
n_estimators                                                      100
max_depth                                                          10
criterion                                                     entropy
accuracy                                                          NaN
Accuracy                                                       0.2625
Precision                                                     0.26271
Recall                                                         0.2625
F1 Score                                                     0.262058
Confusion Matrix    [[83, 85, 105, 114], [89, 109, 94, 105], [88, ...
Name: 0, dtype: object


In [47]:
X_full = pd.concat([X_train, X_val])
y_full = pd.concat([y_train, y_val])



In [48]:

best_model = RandomForestClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    criterion=best_params['criterion'],
    random_state=42
)
best_model.fit(X_full, y_full)

RandomForestClassifier(criterion='entropy', max_depth=10, random_state=42)

In [49]:
y_pred = best_model.predict(X_test)
print("\nEvaluación final en el conjunto de test:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))


Evaluación final en el conjunto de test:
Accuracy: 0.2255
Reporte de clasificación:
                precision    recall  f1-score   support

 Chemotherapy       0.21      0.15      0.18       499
Immunotherapy       0.23      0.21      0.22       511
    Radiation       0.24      0.29      0.26       500
      Surgery       0.22      0.25      0.23       490

     accuracy                           0.23      2000
    macro avg       0.22      0.23      0.22      2000
 weighted avg       0.22      0.23      0.22      2000



In [ ]:
nuevo_dict = {col: [0.0] for col in X_full.columns}
nuevo_dict.update({
    'Age': [63],
    'Gender': [1],
    'Smoking_Status': [2],
    'Weight': [72],
    'Height': [170],
    'Treatment_Delay': [15],
    'Nationality_Expatriate': [1],
    'Cancer_Type_Prostate': [1],
    'Cancer_Stage_III': [1],
    'Outcome_Under Treatment': [1],
    'Ethnicity_Arab': [0]
})

In [51]:

new_df = pd.DataFrame(nuevo_dict)[X_full.columns]
new_df = pd.DataFrame(imputer.transform(new_df), columns=X_full.columns)

In [54]:
predicted_treatment = best_model.predict(new_df)
print("\nTratamiento predicho para el nuevo paciente:", predicted_treatment[0])


Tratamiento predicho para el nuevo paciente: Radiation


In [55]:
joblib.dump(best_model, 'random_forest_model.pkl')
print("\n Modelo guardado exitosamente como 'random_forest_model.pkl'")


 Modelo guardado exitosamente como 'random_forest_model.pkl'


In [ ]:
# Cargar modelo guardado y verificar predicción
loaded_model = joblib.load('random_forest_model.pkl')
print("\n Modelo cargado correctamente desde archivo")


✅ Modelo cargado correctamente desde archivo


In [ ]:
predicted_loaded = loaded_model.predict(new_df)
print("\n Predicción usando el modelo cargado:", predicted_loaded[0])


✅ Predicción usando el modelo cargado: Radiation


Después de evaluar múltiples combinaciones de hiperparámetros para el modelo **Random Forest**, se seleccionó la mejor configuración con base en la métrica *F1 Score* utilizando el conjunto de validación.

Posteriormente, el modelo fue reentrenado con la combinación de los conjuntos de entrenamiento y validación para aprovechar toda la información disponible y se evaluó en el conjunto de prueba (`test`), obteniendo las siguientes métricas:

- **Accuracy en test:** 0.22 (aproximadamente)
- **Desempeño general:** Aceptable, considerando la complejidad del problema y la naturaleza de los datos clínicos.

Además, se utilizó el modelo entrenado para **realizar una predicción sobre un nuevo paciente** con las siguientes características:

- Edad: 63 años  
- Sexo: Masculino  
- Estado de fumador: Fumador actual  
- Tipo de cáncer: Próstata  
- Etapa del cáncer: III  
- Tiempo de espera entre diagnóstico y tratamiento: 15 días  
- Nacionalidad: Extranjero  
- Resultado clínico: En tratamiento  
- Grupo étnico: Otro (no árabe)

El modelo predijo el **tipo de tratamiento más probable** para este paciente como:

> **Tratamiento predicho:** `Radiation` *(por ejemplo, según el resultado devuelto)*

Este resultado demuestra la utilidad práctica del modelo para apoyar decisiones clínicas basadas en datos históricos, ofreciendo una herramienta adicional para los profesionales de la salud en la personalización del tratamiento.

---
